# IBM Applied Data Science Capstone Course by Coursera
### Week 5 Final Report
**_Opening a New Gym in St.Louis, Missouri_**
- Build a dataframe of neighborhoods in St.Louis, Missouri by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new gym
***
### 1. Import libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame

In [3]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighborhoods_in_St._Louis").text

In [4]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [5]:
# create a list to store neighborhood data
neighborhoodList = []

In [6]:
# append the data into the list
for row in soup.find_all("div", class_="mw-content-ltr")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [7]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,"► Central West End, St. Louis‎ (17 P)"
1,► Downtown St. Louis‎ (48 P)
2,"► Downtown West, St. Louis‎ (19 P)"
3,► Midtown St. Louis‎ (20 P)
4,List of neighborhoods of St. Louis


In [8]:
# print the number of rows of the dataframe
kl_df.shape

(93, 1)

### 3. Get the geographical coordinates


In [9]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, St.Louis, Missouri'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [10]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [11]:
coords

[[38.64170000000007, -90.25031999999999],
 [38.65340003664378, -90.19202996335623],
 [38.62975000000006, -90.20624999999995],
 [38.62736000000007, -90.22266999999994],
 [38.67032993782109, -90.20125004053882],
 [38.6772799233944, -90.50661997388784],
 [38.70553000000007, -90.23000999999994],
 [38.597660000000076, -90.23096999999996],
 [38.59962000000007, -90.21887999999996],
 [38.58386000000007, -90.26637999999997],
 [38.620960000000025, -90.25062999999994],
 [38.62775000000005, -90.19955999999996],
 [38.562020000000075, -90.27722999999997],
 [38.55639000000008, -90.26642999999996],
 [38.63901000000004, -90.19948999999997],
 [38.64170000000007, -90.25031999999999],
 [38.62733000000003, -90.27975999999995],
 [38.673439948352666, -90.34664002726596],
 [38.626990000000035, -90.29080999999996],
 [38.61140000000006, -90.29169999999993],
 [38.674000000000035, -90.20877999999999],
 [38.63691000000006, -90.18941999999998],
 [38.61298000000005, -90.23625999999996],
 [38.649470000000065, -90.278

In [12]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [13]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [14]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(93, 3)


,Neighborhood,Latitude,Longitude
0,"► Central West End, St. Louis‎ (17 P)",38.641700,-90.250320
1,► Downtown St. Louis‎ (48 P),38.653400,-90.192030
2,"► Downtown West, St. Louis‎ (19 P)",38.629750,-90.206250
3,► Midtown St. Louis‎ (20 P),38.627360,-90.222670
4,List of neighborhoods of St. Louis,38.670330,-90.201250
5,"Academy, St. Louis",38.677280,-90.506620
6,"Baden, St. Louis",38.705530,-90.230010
7,"Benton Park West, St. Louis",38.597660,-90.230970
8,"Benton Park, St. Louis",38.599620,-90.218880
9,"Bevo Mill, St. Louis",38.583860,-90.266380


In [18]:
kl_df = kl_df.drop([0, 1, 2, 3, 4])
kl_df.index = range(len(kl_df))
print(kl_df.shape)
kl_df

(88, 3)


,Neighborhood,Latitude,Longitude
0,"Academy, St. Louis",38.677280,-90.506620
1,"Baden, St. Louis",38.705530,-90.230010
2,"Benton Park West, St. Louis",38.597660,-90.230970
3,"Benton Park, St. Louis",38.599620,-90.218880
4,"Bevo Mill, St. Louis",38.583860,-90.266380
5,"Botanical Heights, St. Louis",38.620960,-90.250630
6,"The Bottle District, St Louis",38.627750,-90.199560
7,"Boulevard Heights, St. Louis",38.562020,-90.277230
8,"Carondelet, St. Louis",38.556390,-90.266430
9,"Carr Square, St. Louis",38.639010,-90.199490


In [19]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

### 4. Create a map of boone county with neighborhoods superimposed on top

In [20]:
# get the coordinates of St.Louis,Missouri
address = 'St.Louis, Missouri'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of St.Louis, Missouri {}, {}.'.format(latitude, longitude))

The geograpical coordinate of St.Louis, Missouri 38.6529545, -90.24111656024635.


In [22]:
# create map of St.Louis using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  

map_kl

In [23]:
# save the map as HTML file
map_kl.save('map_kl.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [24]:
# define Foursquare Credentials and Version
CLIENT_ID = '2XCVRUJA01L0OZEWDRNAME3TMTFNONQ2CUBCD4IIZ3ACROAG' # your Foursquare ID
CLIENT_SECRET = 'KPFFEIV5LAYZRTDHM3AOVSHURI0YXVKIGHKXRHTRPUWK11BH' # your Foursquare Secret
VERSION = '20200401' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2XCVRUJA01L0OZEWDRNAME3TMTFNONQ2CUBCD4IIZ3ACROAG
CLIENT_SECRET:KPFFEIV5LAYZRTDHM3AOVSHURI0YXVKIGHKXRHTRPUWK11BH


**Now, let's get the top 100 venues that are within a radius of 2000 meters.**

In [25]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [26]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6769, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"Academy, St. Louis",38.67728,-90.50662,McArthurs Bakery,38.679504,-90.506024,Bakery
1,"Academy, St. Louis",38.67728,-90.50662,Paul Manno's,38.680186,-90.499845,Italian Restaurant
2,"Academy, St. Louis",38.67728,-90.50662,Kim Cheese,38.680299,-90.498243,Korean Restaurant
3,"Academy, St. Louis",38.67728,-90.50662,Dierbergs,38.679487,-90.500365,Grocery Store
4,"Academy, St. Louis",38.67728,-90.50662,Saint Louis Bread Co.,38.679104,-90.502683,Bakery


**Let's check how many venues were returned for each neighorhood**

In [27]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"Academy, St. Louis",40,40,40,40,40,40
"Baden, St. Louis",11,11,11,11,11,11
"Benton Park West, St. Louis",100,100,100,100,100,100
"Benton Park, St. Louis",100,100,100,100,100,100
"Bevo Mill, St. Louis",100,100,100,100,100,100
"Botanical Heights, St. Louis",100,100,100,100,100,100
"Boulevard Heights, St. Louis",74,74,74,74,74,74
"Carondelet, St. Louis",68,68,68,68,68,68
"Carr Square, St. Louis",100,100,100,100,100,100


**Let's find out how many unique categories can be curated from all the returned venues**

In [28]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 247 uniques categories.


In [29]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Bakery', 'Italian Restaurant', 'Korean Restaurant',
       'Grocery Store', 'Fast Food Restaurant', 'Falafel Restaurant',
       'Thai Restaurant', 'Sandwich Place', 'Chinese Restaurant',
       'Lingerie Store', 'Greek Restaurant', 'Café', 'Bar', 'Coffee Shop',
       'Pizza Place', 'Salon / Barbershop', 'Shipping Store', 'Diner',
       'Thrift / Vintage Store', 'Gym', 'Mexican Restaurant',
       'Convenience Store', 'Automotive Shop', 'Bubble Tea Shop',
       'American Restaurant', 'Pet Store', 'New American Restaurant',
       'Home Service', 'Smoke Shop', 'Football Stadium', 'Food', 'Lounge',
       'Discount Store', 'Fried Chicken Joint', 'Waste Facility', 'Park',
       'Art Gallery', 'Whisky Bar', 'Tea Room', 'Boutique', 'BBQ Joint',
       'Brewery', 'Asian Restaurant', 'Middle Eastern Restaurant',
       'Ethiopian Restaurant', 'Vietnamese Restaurant',
       'Mediterranean Restaurant', 'Taco Place',
       'Vegetarian / Vegan Restaurant', 'Filipino Restaurant'],
  

In [44]:
# check if the results contain "Gym"
"Neighborhood" in venues_df['VenueCategory'].unique()

False

### 6. Analyze Each Neighborhood

In [31]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(6769, 248)


,Neighborhoods,ATM,Accessories Store,Advertising Agency,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Casino,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Basketball Court,College Rec Center,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Massage Studio,Medical School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,National Park,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pawn Shop,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Pool Hall,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Trail,Train,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Waste Facility,Water Park,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,"Academy, St. Louis",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Academy, St. Louis

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [32]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(88, 248)


,Neighborhoods,ATM,Accessories Store,Advertising Agency,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Casino,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Basketball Court,College Rec Center,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Massage Studio,Medical School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,National Park,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pawn Shop,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Pool Hall,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Trail,Train,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Waste Facility,Water Park,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,"Academy, St. Louis",0.000000,0.000000,0.000000,0.025000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.025000,0.000000,0.000000,0.050000,0.000000,0.025000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.025,0.000000,0.000000,0.00,0.000000,0.025000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.025000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

In [51]:
len(kl_grouped[kl_grouped["Gym"] > 0])

46

In [52]:
kl_mall = kl_grouped[["Neighborhoods","Gym"]]

In [53]:
kl_mall.head()

,Neighborhoods,Gym
0,"Academy, St. Louis",0.025
1,"Baden, St. Louis",0.000
2,"Benton Park West, St. Louis",0.000
3,"Benton Park, St. Louis",0.000
4,"Bevo Mill, St. Louis",0.010


### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in St.Louis into 3 clusters.

In [54]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 0, 0, 0, 1, 2, 2, 0, 0], dtype=int32)

In [55]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [56]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Gym,Cluster Labels
0,"Academy, St. Louis",0.025,1
1,"Baden, St. Louis",0.000,0
2,"Benton Park West, St. Louis",0.000,0
3,"Benton Park, St. Louis",0.000,0
4,"Bevo Mill, St. Louis",0.010,0


In [57]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(88, 5)


,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
0,"Academy, St. Louis",0.025,1,38.67728,-90.50662
1,"Baden, St. Louis",0.000,0,38.70553,-90.23001
2,"Benton Park West, St. Louis",0.000,0,38.59766,-90.23097
3,"Benton Park, St. Louis",0.000,0,38.59962,-90.21888
4,"Bevo Mill, St. Louis",0.010,0,38.58386,-90.26638


In [58]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(88, 5)


,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
43,LaSalle Park,0.000000,0,38.615340,-90.202340
57,"North Riverfront, St. Louis",0.000000,0,38.699250,-90.218830
56,"North Point, St. Louis",0.000000,0,38.715720,-90.246340
54,"Near North Riverfront, St. Louis",0.000000,0,38.658250,-90.189250
53,"Mount Pleasant, St. Louis",0.000000,0,38.574520,-90.236920
52,Midtown St. Louis,0.010000,0,38.627360,-90.222670
51,"McKinley Heights, St. Louis",0.000000,0,38.610380,-90.217220
58,"O'Fallon, St. Louis",0.000000,0,38.675560,-90.224420
50,"Mark Twain/I-70 Industrial, St. Louis",0.000000,0,38.687720,-90.267100
48,"Marine Villa, St. Louis",0.000000,0,38.585980,-90.219800


In [59]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [60]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [61]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
43,LaSalle Park,0.000000,0,38.615340,-90.202340
57,"North Riverfront, St. Louis",0.000000,0,38.699250,-90.218830
56,"North Point, St. Louis",0.000000,0,38.715720,-90.246340
54,"Near North Riverfront, St. Louis",0.000000,0,38.658250,-90.189250
53,"Mount Pleasant, St. Louis",0.000000,0,38.574520,-90.236920
52,Midtown St. Louis,0.010000,0,38.627360,-90.222670
51,"McKinley Heights, St. Louis",0.000000,0,38.610380,-90.217220
58,"O'Fallon, St. Louis",0.000000,0,38.675560,-90.224420
50,"Mark Twain/I-70 Industrial, St. Louis",0.000000,0,38.687720,-90.267100
48,"Marine Villa, St. Louis",0.000000,0,38.585980,-90.219800


#### Cluster 1

In [62]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
61,"Patch, St. Louis",0.032258,1,38.543530,-90.261500
75,"The Hill, St. Louis",0.020000,1,38.618310,-90.276030
12,"Clayton/Tamm, St. Louis",0.020000,1,38.626990,-90.290810
5,"Botanical Heights, St. Louis",0.030000,1,38.620960,-90.250630
81,"Visitation Park, St. Louis",0.020000,1,38.656620,-90.275610
85,"Wells/Goodfellow, St. Louis",0.020408,1,38.677410,-90.270340
10,"Cheltenham, St. Louis",0.026316,1,38.627330,-90.279760
70,"Southwest Garden, St. Louis",0.020000,1,38.607520,-90.275600
0,"Academy, St. Louis",0.025000,1,38.677280,-90.506620
13,"Clifton Heights, St. Louis",0.030000,1,38.611400,-90.291700


#### Cluster 2

In [65]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
34,"Hamilton Heights, St. Louis",0.047619,2,38.66876,-90.27993
6,"Boulevard Heights, St. Louis",0.054054,2,38.56202,-90.27723
7,"Carondelet, St. Louis",0.044118,2,38.55639,-90.26643
36,"Holly Hills, St. Louis",0.055556,2,38.56852,-90.26138


,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
